## import Libraries

In [1]:
import os
import geopandas as gpd
from shapely.geometry import Point,Polygon,box
from pyproj import CRS
from numpy import arange
import pandas as pd

### reading "road.shp" File and re-projecting it to (epsg:5179)

In [2]:
road = gpd.read_file(r"inputs\road.shp",encoding="utf8")
road.to_crs(epsg=5179,inplace=True)
road.head(3)

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,LENGTH,REMARK,geometry
0,3880778900,3880289100,3880289500,1,107,000,-,금오14길,0,0,000,40,0,0,0,134.2642,None,"LINESTRING (1138704.411 1703375.173, 1138719.4..."
1,3880779000,3880289500,3880289000,1,107,000,-,금오14길,0,0,000,40,0,0,0,40.1724,None,"LINESTRING (1138719.394 1703508.432, 1138724.4..."
2,3880779100,3880289000,3880289500,1,107,000,-,금오14길,0,0,000,40,0,0,0,40.1723,None,"LINESTRING (1138712.566 1703549.774, 1138707.4..."


### reading "camera.csv" file

In [3]:
cameras = gpd.read_file(r"inputs\camera.csv",driver="csv",encoding="utf8")

cameras.head(3)

##cameras is a dataframe ↓↓ 

,ID,City,Direction,Angle,Latitude,Longitude,Speed Limit,R_code,geometry
0,F8863,수원시,3,W+E,37.265934,126.957433,60,2,None
1,F7559,용인시,1,SE,37.321687,127.079906,50,2,None
2,F7569,부천시,1,N,37.518766,126.770096,30,2,None


### convert "cameras" to GeoDataFrame

In [4]:
cameras["Longitude"]=cameras["Longitude"].astype(float)
cameras["Latitude"]=cameras["Latitude"].astype(float)
cameras["geometry"]=[Point(xy) for xy in zip(cameras["Longitude"],cameras["Latitude"])]
cameras.crs = CRS.from_epsg(4326).to_wkt() ## cameras is already georeferenced using WGS84

## re-projecting cameras to (epsg:5179)

In [6]:
cameras.to_crs(epsg=5179,inplace=True)
cameras.head(3)

,ID,City,Direction,Angle,Latitude,Longitude,Speed Limit,R_code,geometry
0,F8863,수원시,3,W+E,37.265934,126.957433,60,2,POINT (951892.973 1918696.861)
1,F7559,용인시,1,SE,37.321687,127.079906,50,2,POINT (962779.653 1924826.835)
2,F7569,부천시,1,N,37.518766,126.770096,30,2,POINT (935499.426 1946858.495)


### writing "Sq_Grid" function to create squares grid to cover road

In [7]:
def Sq_Grid(gdf,length,epsg_code):
    cells=list()
    minx,miny,maxx,maxy = gdf.total_bounds
    for x0 in arange(minx,maxx,length):  ##x0 is a minimum x axis to box
        for y1 in arange(maxy,miny,-length):  ##y1 is a maximum y axis to box
            x1=x0+length  ##x1 is a maximum x axis to box
            y0=y1-length  ##y0 is a minimum y axis to box
            cells.append(box(x0,y0,x1,y1))
    return gpd.GeoDataFrame(geometry=cells,crs=CRS.from_epsg(epsg_code))

'''
gdf : GeoDataframe
length : side square length
epsg_code : code to crs
----------------------------------
The box is ↓↓↓

              
    (x0,y1)_______ (x1,x2)
          |       |
          |       |  
          |__ ____|
    (x0,y0)        (x1,y0)

x0=minx
y0=miny
x1=maxx
x2=maxy
--------------------------------
use np.arange because float values

'''

grid = Sq_Grid(road,20000,5179)
print(grid.head(3))

                                            geometry
0  POLYGON ((876040.080 2045187.046, 876040.080 2...
1  POLYGON ((876040.080 2025187.046, 876040.080 2...
2  POLYGON ((876040.080 2005187.046, 876040.080 2...


### create "grid_id" column

In [8]:
grid["grid_id"]=[ID for ID in range(1,len(grid)+1)]
print(grid.head(3))

                                            geometry  grid_id
0  POLYGON ((876040.080 2045187.046, 876040.080 2...        1
1  POLYGON ((876040.080 2025187.046, 876040.080 2...        2
2  POLYGON ((876040.080 2005187.046, 876040.080 2...        3


### the intersection between road and grid

In [9]:
road_intersection=gpd.overlay(road,grid,how='intersection')
print(road_intersection.head(3).to_markdown())
## the "grid_id" is create in roads by overlay analysis

|    |    LINK_ID |     F_NODE |     T_NODE |   LANES |   ROAD_RANK |   ROAD_TYPE | ROAD_NO   | ROAD_NAME   |   ROAD_USE |   MULTI_LINK |   CONNECT |   MAX_SPD |   REST_VEH |   REST_W |   REST_H |   LENGTH | REMARK   |   grid_id | geometry                                                                              |
|---:|-----------:|-----------:|-----------:|--------:|------------:|------------:|:----------|:------------|-----------:|-------------:|----------:|----------:|-----------:|---------:|---------:|---------:|:---------|----------:|:--------------------------------------------------------------------------------------|
|  0 | 3880778900 | 3880289100 | 3880289500 |       1 |         107 |         000 | -         | 금오14길    |          0 |            0 |       000 |        40 |          0 |        0 |        0 | 134.264  |          |       439 | LINESTRING (1138704.411147075 1703375.173355783, 1138719.404664226 1703508.519328477) |
|  1 | 3880779000 | 3880289500 | 3880289000 | 

### create "grid_id" column in cameras

In [10]:
cameras_grid_id=gpd.sjoin(cameras,grid,how="left",op="within",rsuffix='grid',)

'''
how="left" ↓↓

a result of "left" same a result of "inner"

I used "left" to save index ascending of cameras

'''
print(cameras_grid_id.head(3).to_markdown())

|    | ID    | City   |   Direction | Angle   |   Latitude |   Longitude |   Speed Limit |   R_code | geometry                                    |   index_grid |   grid_id |
|---:|:------|:-------|------------:|:--------|-----------:|------------:|--------------:|---------:|:--------------------------------------------|-------------:|----------:|
|  0 | F8863 | 수원시 |           3 | W+E     |    37.2659 |     126.957 |            60 |        2 | POINT (951892.9733604998 1918696.861407467) |          127 |       128 |
|  1 | F7559 | 용인시 |           1 | SE      |    37.3217 |     127.08  |            50 |        2 | POINT (962779.6532611722 1924826.834853379) |          157 |       158 |
|  2 | F7569 | 부천시 |           1 | N       |    37.5188 |     126.77  |            30 |        2 | POINT (935499.4257011133 1946858.495053298) |           95 |        96 |


### create grid boundary

In [11]:
grid["geometry"] = grid["geometry"].boundary
print(grid.head(3).to_markdown())

|    | geometry                                                                                                                                                                                             |   grid_id |
|---:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------:|
|  0 | LINESTRING (876040.0799623571 2045187.046464815, 876040.0799623571 2065187.046464815, 856040.0799623571 2065187.046464815, 856040.0799623571 2045187.046464815, 876040.0799623571 2045187.046464815) |         1 |
|  1 | LINESTRING (876040.0799623571 2025187.046464815, 876040.0799623571 2045187.046464815, 856040.0799623571 2045187.046464815, 856040.0799623571 2025187.046464815, 876040.0799623571 2025187.046464815) |         2 |
|  2 | LINESTRING (876040.0799623571 2005187.046464815, 876040.0799623571 2025187.046464815, 856040.0799623571 2025187.046464815

### create otuputs directory

In [12]:
path = os.path.join(os.getcwd(),"otuputs")
os.mkdir(path)

### create boundary,cameras and roads geojson files in otuputs dir

In [13]:
for ID in grid["grid_id"]:
    os.mkdir(os.path.join("otuputs",str(ID)))
    grid[grid["grid_id"]==ID].to_file(os.path.join("otuputs",str(ID),"boundary.geojson"),driver="GeoJSON")
    if (cameras_grid_id[cameras_grid_id["grid_id"]==ID].empty):
        open(os.path.join("otuputs",str(ID),"cameras.geojson"),"x")
    else:
        cameras_grid_id[cameras_grid_id["grid_id"]==ID].to_file(os.path.join("otuputs",str(ID),"cameras.geojson"),driver="GeoJSON",encoding="utf-8")
    
    if (road_intersection[road_intersection["grid_id"]==ID].empty):
        open(os.path.join("otuputs",str(ID),"roads.geojson"),"x")
    else:
        road_intersection[road_intersection["grid_id"]==ID].to_file(os.path.join("otuputs",str(ID),"roads.geojson"),driver="GeoJSON",encoding="utf-8")